# Homework 3

In [1]:
 using Distributions, Plots, Interact, BenchmarkTools

WebIO._IJuliaInit()

## Consider the bivariate normal distribution we introduced in the class. Let's make the assumptions that  $\mu_x = 0$, $\mu_y = \mu_y$, $\sigma_x = \sigma_x$, $\sigma_y = 1$, and $\rho=\rho$.

### Write down the conditional distributions of X and Y. Note that both of them would be normal distributions. (See Section 1.4.3 and 1.4.3.1 of the lecture note.)

\begin{align}
(X|Y=y) & \sim N\left( \rho \sigma_x(y-\mu_y),\quad \sigma_x^2(1-\rho^2) \right),\\
(Y|X=x) & \sim N\left( \mu_y + \rho \frac{x}{\sigma_x},\quad (1-\rho^2) \right).
\end{align}

### Write a Julia function of Gibbs sampler that allows users to choose: the number of sampling points $N$, the values of $\mu_y$, $\sigma_x$, $\rho$, and initial values of $(x_0, y_0)$, and the number of burn-in points. You should follow the steps in Section 1.3 of the lecture note to draw $(x_n, y_n), n=1,\ldots,N$.


In [2]:
drawX_conditional_onY(σₓ, μy, ρ; y) = rand(Normal(ρ*σₓ*(y-μy), σₓ*sqrt(1-ρ^2)))
drawY_conditional_onX(σₓ, μy, ρ; x) = rand(Normal(μy+ρ*(x/σₓ), sqrt(1-ρ^2)))

function gibbs_sample(;σₓ, μy, ρ, n=100_000_000, burn=10_000, init=[2, 2])
    -1<= ρ <= 1 ||throw("Such correlation coefficient does not exist.") # 曾勁松
    N = 1 + burn + n  # 劉浩揚
    x = ones(N) * init[1]
    y = ones(N) * init[2]
    for i = 2:N
        x[i] = drawX_conditional_onY(σₓ, μy, ρ, y=y[i-1])
        y[i] = drawY_conditional_onX(σₓ, μy, ρ, x=x[i])
    end
    return x[2+burn:end], y[2+burn:end]
end;

In [3]:
# always check the result
x, y = gibbs_sample(σₓ=0.6, μy=2., ρ=0.7)

@show [mean(x), mean(y)]
@show [std(x), std(y)]
@show cor(x, y);

[mean(x), mean(y)] = [-8.5456026300886e-5, 1.999795933582657]
[std(x), std(y)] = [0.6000801151687235, 1.0001338072404822]
cor(x, y) = 0.6999878344247872


In [4]:
mutable struct GibbsParam{T, S, U, V, W, X}
    μₓ::T
    σₓ::S
    μy::U
    σy ::V
    ρ::W
    n::Int64
    burn::Int64
    init::X
end

# outer constructor
function GibbsParam(;σₓ, μy, ρ, n=100_000_000, burn=10_000, init=[2,2])
    -1 <= ρ <= 1 || error("Such correlation coefficient does not exist.")  # 曾勁松
    return GibbsParam(0., σₓ, μy, 1., ρ, n, burn, init)
end

# multiple dispatch
drawX_conditional_onY(p::GibbsParam; y) = drawX_conditional_onY(p.σₓ, p.μy, p.ρ; y=y)
drawY_conditional_onX(p::GibbsParam; x) = drawY_conditional_onX(p.σₓ, p.μy, p.ρ; x=x)

function gibbs_sample(p::GibbsParam)
    # n, burn, init = p.n, p.burn, p.init
    N = 1 + p.burn + p.n
    x = ones(N) * p.init[1]
    y = ones(N) * p.init[2]
    for i = 2:N
        x[i] = drawX_conditional_onY(p, y=y[i-1])
        y[i] = drawY_conditional_onX(p, x=x[i])
    end
    
    return x[2+p.burn:end], y[2+p.burn:end]
end;

In [5]:
p = GibbsParam(σₓ=0.6, μy=2., ρ=0.7)
x, y = gibbs_sample(p)
    
@show [mean(x), mean(y)]
@show [std(x), std(y)]
@show cor(x, y);

[mean(x), mean(y)] = [-5.794289710269368e-5, 1.999958203018471]
[std(x), std(y)] = [0.600070683230665, 1.000081605182953]
cor(x, y) = 0.7000164656654142


In [6]:
@btime gibbs_sample(GibbsParam(σₓ=0.9, μy=2., ρ=0.7))
@btime gibbs_sample(σₓ=0.9, μy=2., ρ=0.7);

  4.756 s (14 allocations: 4.47 GiB)
  5.977 s (13 allocations: 4.47 GiB)


### Draw a graph  similar to the one we showed in the class. Remember to add sliders for both $N$ and $\rho$.

In [7]:
# 潘家栩
@manipulate for n in 50:50:5000, corr in -0.99:0.01:0.99
    # x, y = gibbs_sample(σₓ=0.9, μy=2., ρ=corr, n=n)
    p.n, p.ρ = n, corr
    x, y = gibbs_sample(p)

    layout = @layout [a{0.6w,0.4h} _
                      b{0.6w,0.6h} c{0.4w, 0.6h}]
    default(legend=false)
    plot(layout=layout, link=:both, size=(500, 500), margin=-2Plots.pt)

    scatter!(
        x, y,
        markersize=1, markercolor=:red, markerstrokecolor=:red,
        framestyle=:box, aspect_ratio=:equal,
        xticks=-5:5, yticks=-5:5, xlabel="X", ylabel="Y",
        xlim=(-5, 5), ylim=(-3, 7), subplot=2
    )

    histogram!(
        [x y], 
        subplot=[1 3], color=:lightgrey, normalize=true, bins = min(100, n), 
        orientation=[:v :h], framestyle=:none, grid=false
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["n"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 100, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(50), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\pupss\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\pupss\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\pupss\\.julia\\packages\\InteractBase\\Qhvxg\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\pupss\\.julia\\packages\\InteractBase\\Qhvxg\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\pupss\\.julia\\packages\\Interact\\PENUy\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000072c0cbb0, Task (runnable) @0x0000000072c0cbb0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"50\",\"100\",\"150\",\"200\",\"250\",\"300\",\"350\",\"400\",\"450\",\"500\",\"550\",\"600\",\"650\",\"700\",\"750\",\"800\",\"850\",\"900\",\"950\",\"1000\",\"1050\",\"1100\",\"1150\",\"1200\",\"1250\",\"1300\",\"1350\",\"1400\",\"1450\",\"1500\",\"1550\",\"1600\",\"1650\",\"1700